### IPO price scraper - v2.0

In this script, I will scrape the stock info and company profile from Yahoo Finance for the IPO companies scraped in the previous step from NASDAQ site (see IPO_scraper.ipynb)

This is the second version of the Yahoo Finance scraper. I changed the way the data is scraped - it now scrapes the trading prices only if the IPO date from Nasdaq data frame corresponds to date in Yahoo Finance. It turns out that by accessing the first value in Yahoo Finance (as in version 1.0) provides wrong results for some companies, because Yahoo Finance keep record for them longer than IPO priced date (why??).

As a result, this scraper yielded more notscraped tickers than the scraper 1.0 <br>
v 1.0 - From 1569 IPO companies listed on Nasdaq, 433 companies were not scraped on Yahoo Finance<br>
v 2.0 - From 1533 IPO companies listed on Nasdaq, 703 companies were not scraped on Yahoo Finance<br>

When running a month later, there was also more "misalignment between company name and its ticker" (no code changed), so instead of 1600 scraped from Nasdaq, after all the checks we got 1533 companies.<br>

Example of nonscraped ticker is, e.g. 'MUSA'. The company was priced on NASDAQ on 2010-04-09. But the earliest date on Yahoo Finance is 2013-08-19.<br>


<img src="assets/MUSA_example.png" alt="drawing" height="1500" width="800"/>

The rest of the script is the same as in v1.0.

In [2]:
import re
from difflib import SequenceMatcher
import pandas_datareader.data as pdr #Yahoo Finance ticker
import pandas as pd
import numpy as np

from datetime import datetime

from bs4 import BeautifulSoup
import requests
from time import sleep

import dill

In [3]:
#run this cell if want to import dill from earlier session
#dill.load_session('dill/ipo_price_scraper.pkl')

In [4]:
df = pd.read_csv('data/ipo_clean_2010_2018.csv', parse_dates=['Date Priced', 'year'])
df.shape

(1600, 13)

In [5]:
df.dtypes

Company Name            object
Symbol                  object
Market                  object
Price                  float64
Shares                 float64
Offer Amount           float64
Date Priced     datetime64[ns]
employees              float64
address                 object
US_state                object
descriptions            object
link_nasdaq             object
year            datetime64[ns]
dtype: object

## Checking for duplicated tickers and company names:

In [6]:
print("number of duplicated companies in the dataset:", df['Company Name'].duplicated().sum())

number of duplicated companies in the dataset: 3


In [7]:
df[df['Company Name'].duplicated(keep=False)]

,Company Name,Symbol,Market,Price,Shares,Offer Amount,Date Priced,employees,address,US_state,descriptions,link_nasdaq,year
85,AMBOW EDUCATION HOLDING LTD.,AMBO,New York Stock Exchange,10.00,10677207.0,106772070.0,2010-08-05,10361.0,"12TH FLOOR, NO. 1 FINANCIAL STREETCHANG AN CEN...",NaN,We are a leading national provider of educatio...,https://www.nasdaq.com/markets/ipos/company/am...,2010-01-01
205,SANDRIDGE ENERGY INC,SDT,New York Stock Exchange,21.00,15000000.0,315000000.0,2011-04-07,2192.0,"123 ROBERT S. KERR AVENUEOKLAHOMA CITY, OK 731...",OK,SandRidge Mississippian Trust I is a Delaware ...,https://www.nasdaq.com/markets/ipos/company/sa...,2011-01-01
254,SANDRIDGE ENERGY INC,PER,New York Stock Exchange,18.00,30000000.0,540000000.0,2011-08-11,2192.0,"123 ROBERT S. KERR AVENUEOKLAHOMA CITY, OK 731...",OK,SandRidge Permian Trust is a Delaware statutor...,https://www.nasdaq.com/markets/ipos/company/sa...,2011-01-01
341,SANDRIDGE ENERGY INC,SDR,New York Stock Exchange,21.00,26000000.0,546000000.0,2012-04-18,NaN,"123 ROBERT S. KERR AVENUEOKLAHOMA CITY, OK 731...",OK,SandRidge Mississippian Trust II is a Delaware...,https://www.nasdaq.com/markets/ipos/company/sa...,2012-01-01
1503,AMBOW EDUCATION HOLDING LTD.,AMBO,NYSE MKT,4.25,1800000.0,7650000.0,2018-06-01,2657.0,"12TH FLOOR, NO. 1 FINANCIAL STREETCHANG AN CEN...",NaN,"Our mission is to provide Better Schools, Bett...",https://www.nasdaq.com/markets/ipos/company/am...,2018-01-01


Commentary: Ambow Education LTD was enlisted twice. The first one it was liquedated after a scandal in 2014 [1]
Sandridge Energy Inc - all three are different trusts that belong to one company. Technically they are different. 

1. https://ir.theice.com/press/press-releases/nyse-regulation/2014/nyse-to-immediately-suspend-trading-in-american-depository

In [8]:
print("number of duplicated tickers in the dataset:", df['Symbol'].duplicated().sum())

number of duplicated tickers in the dataset: 7


In [9]:
df.iloc[:, :7][df['Symbol'].duplicated(keep=False)]

,Company Name,Symbol,Market,Price,Shares,Offer Amount,Date Priced
60,"HIGHER ONE HOLDINGS, INC.",ONE,New York Stock Exchange,12.00,9000000.0,108000000.0,2010-06-17
85,AMBOW EDUCATION HOLDING LTD.,AMBO,New York Stock Exchange,10.00,10677207.0,106772070.0,2010-08-05
97,SEACUBE CONTAINER LEASING LTD.,BOX,New York Stock Exchange,10.00,9500000.0,95000000.0,2010-10-28
293,BAZAARVOICE INC,BV,NASDAQ,12.00,9484296.0,113811552.0,2012-02-24
331,ACQUITY GROUP LTD,AQ,American Stock Exchange,6.00,5555556.0,33333336.0,2012-04-27
356,"TIAA FSB HOLDINGS, INC.",EVER,New York Stock Exchange,10.00,19220000.0,192200000.0,2012-05-03
715,QUOTIENT TECHNOLOGY INC.,COUP,New York Stock Exchange,16.00,10500000.0,168000000.0,2014-03-07
948,BOX INC,BOX,New York Stock Exchange,14.00,12500000.0,175000000.0,2015-01-23
1207,COUPA SOFTWARE INC,COUP,NASDAQ Global Select,18.00,7400000.0,133200000.0,2016-10-06
1374,AQUANTIA CORP,AQ,NYSE,9.00,6818000.0,61362000.0,2017-11-03


Commentary: There are companies that either got defunct and their tickers were given to new IPOs (like EVER) or there are companies that went to IPO twice (like AMBOW). Either way, because I will scrape stock data based on tickers from Yahoo Finance, which has information only for the most recent company, I keep here only those companies that went to IPO later. Others, I declare defunct.

In [10]:
defunct_oldticker = df['Symbol'][df['Symbol'].duplicated(keep='last')]

In [11]:
df = df[~df['Symbol'].duplicated(keep='last')]

In [12]:
df.shape

(1593, 13)

## Checking suspicious misalignment between company name and its ticker:

After scraping IPO companies I noticed that there are some misalignments between company names and their tickers. This could be due to the fact that some companies are now defunct and their tickers belong now to other companies. Or maybe the company is still functional but changed its ticker. Or maybe there was mistake when entering the ticker on Nasdaq website.

So in this step, I check if letters in the symbol correspond to company name. If they don't, I construct a dict of suspect companies. In the next step, I check these suspicious tickers on Yahoo Finance, retrieve their company names  and compare them to company names on Nasdaq list.

In [13]:
companies_tocheck = {}
for i, symbol in enumerate(df.Symbol):
    for letter in symbol:
        #print(letter)
        if letter not in df.iloc[i, 0]:
            companies_tocheck.update({df.iloc[i, 0]: symbol})   

In [14]:
print(f"{len(companies_tocheck)} companies have suspicious tickers. Will be checked")

284 companies have suspicious tickers. Will be checked


In [15]:
url_1 = 'https://finance.yahoo.com/quote/%s/'
url_2 = 'profile?p=%s'
unmatched = {}
unmatched_tickers = []

for i, ticker in enumerate(list(companies_tocheck.values())):
    if i%10 == 0:
        print(f"scraping {ticker} - {i+1}/{len(companies_tocheck)}")
    result = requests.get(url_1 % ticker + url_2 % ticker)
    content = result.content

    soup = BeautifulSoup(content)
    data = soup.find_all("h1", {"class":"D(ib) Fz(16px) Lh(18px)"})
    if data:
        txt = str([x.get_text() for x in data]).replace(ticker, "").lower()
        regex = re.compile("\w+")
        txt = " ".join(regex.findall(txt)).replace("corporation", "corp").replace("limited", "ltd")
        name = df['Company Name'][df['Symbol'] == ticker].values[0].lower().replace(",", "").replace(".", "").replace("corporation", "corp").replace("limited", "ltd")
        #checking with SequenceMatcher if two strings match for at least 80%
        if SequenceMatcher(None, txt, name).ratio() < 0.8:
            unmatched.update({name: txt})
            unmatched_tickers.append(ticker)

scraping CTC - 1/284
scraping OINK - 11/284
scraping XNY - 21/284
scraping QIHU - 31/284
scraping FENG - 41/284
scraping RNF - 51/284
scraping ROYT - 61/284
scraping SXE - 71/284
scraping FWM - 81/284
scraping ONTX - 91/284
scraping MYCC - 101/284
scraping CLDN - 111/284
scraping SQBK - 121/284
scraping CERU - 131/284
scraping SGNL - 141/284
scraping ROKA - 151/284
scraping JYNT - 161/284
scraping FLKS - 171/284
scraping RKDA - 181/284
scraping MPSX - 191/284
scraping PAVMU - 201/284
scraping PTGX - 211/284
scraping AMMA - 221/284
scraping GTHX - 231/284
scraping KIDS - 241/284
scraping OPBK - 251/284
scraping ENTX - 261/284
scraping RUBY - 271/284
scraping MSC - 281/284


In [16]:
print(f'there are {len(unmatched_tickers)} unmatched tickers. Will be removed')
unmatched

there are 60 unmatched tickers. Will be removed


{'ifm investments ltd': '',
 'energy corp of america': 'eca marcellus trust i',
 'g-estate liquidation stores inc': 'goldman sachs motif manufacturing revolution etf',
 'tal education group': 'tal education group american de',
 'campus crest communities inc': '',
 'dunxin financial holdings ltd': 'china xiniya fashion ltd am',
 'assembly biosciences inc': 'virgin trains usa llc',
 'lentuo international inc': '',
 'leaf group ltd': 'demand media inc',
 'emergent capital inc': '',
 'mmodal inc': 'medx holdings inc',
 'wright medical group nv': 'taronis technologies inc',
 'box ships inc': '',
 'sandridge energy inc': 'sandridge permian trust',
 'link motion inc': 'mobile inc american deposit',
 'c&j old coinc': 'c j energy services ltd',
 'vereit inc': 'american realty capital propert',
 'aptiv plc': 'delphi technologies plc',
 'east dubuque nitrogen partners lp': 'rentech nitrogen partners l p',
 "annie's inc": '',
 'worldpay inc': 'vantiv inc class a',
 'erickson inc': 'erickson incorp

In [17]:
df = df[~df['Symbol'].isin(unmatched_tickers)]

In [18]:
df.shape

(1533, 13)

In [19]:
df.reset_index(drop=True, inplace=True)

### 1.1. Scraping stock data

In [20]:
df_yahoo = pd.DataFrame()
notscraped = []
for i, ticker in enumerate(df.Symbol):
    try:
        if i%10 == 0:
            print(f"scraping {ticker} - {i+1}/{len(df.Symbol)}")
        
        st = pdr.DataReader(ticker, "yahoo")
        st['indx'] = range(len(st))
        inweek_idx = st['indx'][str(df[df['Symbol'] == ticker]['Date Priced'].values[0])] + 7
        inmonth_idx = st['indx'][str(df[df['Symbol'] == ticker]['Date Priced'].values[0])] + 30
        inyear_idx = st['indx'][str(df[df['Symbol'] == ticker]['Date Priced'].values[0])] + 365
        
        firstday_open = st['Open'][str(df[df['Symbol'] == ticker]['Date Priced'].values[0])]
        firstday_adjclose = st['Adj Close'][str(df[df['Symbol'] == ticker]['Date Priced'].values[0])]
        
        inweek_open = st['Open'][st['indx'] == inweek_idx][0]
        inweek_adjclose = st['Adj Close'][st['indx'] == inweek_idx][0]

        inmonth_open = st['Open'][st['indx'] == inmonth_idx][0]
        inmonth_adjclose = st['Adj Close'][st['indx'] == inmonth_idx][0]
        
        inyear_open = st['Open'][st['indx'] == inyear_idx][0]
        inyear_adjclose = st['Adj Close'][st['indx'] == inyear_idx][0]

        
        ticker_df = pd.DataFrame({ticker: {'firstday_adjclose': firstday_adjclose, 'firstday_open': firstday_open, 
                                           'inweek_adjclose': inweek_adjclose, 'inweek_open': inweek_open, 
                                           'inmonth_adjclose': inmonth_adjclose, 'inmonth_open': inmonth_open, 
                                           'inyear_adjclose': inyear_adjclose,'inyear_open': inyear_open}}).T
        df_yahoo = df_yahoo.append(ticker_df)
    except:
        notscraped.append(ticker)
        continue

scraping AMCF - 1/1533
scraping IRWD - 11/1533
scraping CRU - 21/1533
scraping MITL - 31/1533
scraping TNGN - 41/1533
scraping CHRM - 51/1533
scraping BORN - 61/1533
scraping OXF - 71/1533
scraping PATH - 81/1533
scraping GAGA - 91/1533
scraping TOWR - 101/1533
scraping ZGNX - 111/1533
scraping HHC - 121/1533
scraping QRE - 131/1533
scraping CTP - 141/1533
scraping VELT - 151/1533
scraping GEVO - 161/1533
scraping CSOD - 171/1533
scraping ELLI - 181/1533
scraping TZYM - 191/1533
scraping FENG - 201/1533
scraping WIFI - 211/1533
scraping TAOM - 221/1533
scraping SKUL - 231/1533
scraping UBNT - 241/1533
scraping PACD - 251/1533
scraping MCEP - 261/1533
scraping PRLB - 271/1533
scraping CZR - 281/1533
scraping MM - 291/1533
scraping DWRE - 301/1533
scraping BLOX - 311/1533
scraping FB - 321/1533
scraping TSRO - 331/1533
scraping PANW - 341/1533
scraping ELOQ - 351/1533
scraping MEILU - 361/1533
scraping ANFI - 371/1533
scraping ATOS - 381/1533
scraping LAND - 391/1533
scraping CNOB - 401/

In [21]:
df_yahoo.head()

,firstday_adjclose,firstday_open,inmonth_adjclose,inmonth_open,inweek_adjclose,inweek_open,inyear_adjclose,inyear_open
AMCF,5.770000,6.30,7.590000,8.000000,5.830000,6.030000,2.720000,2.58
GNRC,8.460629,13.00,9.719180,14.910000,8.612183,13.050000,13.415763,20.24
QNST,15.000000,15.00,16.410000,16.010000,12.980000,13.500000,12.740000,12.52
TRNO,14.692022,18.75,15.330121,19.530001,14.770803,18.809999,13.518149,17.02
PDM,9.543714,14.75,12.046385,19.299999,10.400203,17.830000,14.289856,21.02


## Checking companies not scraped in Yahoo Finance:

In this part, I will scrutinize the stocks that were nor parsed on Yahoo Finance. There could be two reasons for this - either the company is defunct (merged/was acquired or otherwise delisted from the stock exchange) or the company is still funcational but changed the ticker. In the latter case, we want to include them into our IPO list and redo the scraping from Yahoo Finance for the new ticker. <br>

To check if the company is defunct, I utilize Seeking Alpha website that provides exactly the info we need.

In [22]:
print(f"From {len(df.Symbol)} IPO companies listed on Nasdaq, {len(notscraped)} companies were not scraped on Yahoo Finance")

From 1533 IPO companies listed on Nasdaq, 699 companies were not scraped on Yahoo Finance


In [ ]:
# defunct = []
# newticker = []
# acquired = []

# url = 'https://seekingalpha.com/symbol/%s'
    
# notscraped_df = pd.DataFrame()
# headers = {'User-Agent' : "non-profit learning project"}

# for ticker in notscraped:
#     print(f'scrap - {url % ticker}')
#     result = requests.get(url % ticker, headers=headers).content
#     sleep(15)
    
#     soup = BeautifulSoup(result)
#     m = soup.find_all("div", {"class": "defunct_message defunct_message_etf"})
#     l = soup.find_all("link", {"rel": "canonical"})
#     if m:
#         print(m)
#         defunct.append(ticker)
#     elif l:
#         print(l)
#         nt = str(l).replace('[<link href="https://seekingalpha.com/symbol/', "").replace('\" rel="canonical"/>]', '')
#         newticker.append(nt)
#     else:
#         acquired.append(ticker)

In [ ]:
# print(f"from {len(df.Symbol)} IPO companies listed on Nasdaq, {len(defunct)} companies are defunct as of May 2019, {len(acquired)} were probably acquired and {len(newticker)} changed their ticker")

In [ ]:
# acquired

In [ ]:
# sorted(defunct)
# df[df['Symbol'].isin(defunct)]

In [ ]:
# newticker[:10]

## Joining two dataframes - from NASDAQ and from Yahoo Finance:

In [23]:
df.index = df.Symbol

In [24]:
df_ipo = df.join(df_yahoo, how='inner')
df_ipo.head()

,Company Name,Symbol,Market,Price,Shares,Offer Amount,Date Priced,employees,address,US_state,...,link_nasdaq,year,firstday_adjclose,firstday_open,inmonth_adjclose,inmonth_open,inweek_adjclose,inweek_open,inyear_adjclose,inyear_open
AMCF,ANDATEE CHINA MARINE FUEL SERVICES CORP,AMCF,NASDAQ,6.3,3134921.0,19750002.0,2010-01-26,128.0,NO. 68 BINHAI RD DALIAN XIGANG DISTRICTDALIAN ...,NaN,...,https://www.nasdaq.com/markets/ipos/company/an...,2010-01-01,5.770000,6.30,7.590000,8.000000,5.830000,6.030000,2.720000,2.58
GNRC,GENERAC HOLDINGS INC.,GNRC,New York Stock Exchange,13.0,18750000.0,243750000.0,2010-02-11,1486.0,"S45 W29290 HIGHWAY 59WAUKESHA, WI 53187",WI,...,https://www.nasdaq.com/markets/ipos/company/ge...,2010-01-01,8.460629,13.00,9.719180,14.910000,8.612183,13.050000,13.415763,20.24
QNST,"QUINSTREET, INC",QNST,NASDAQ,15.0,10000000.0,150000000.0,2010-02-11,568.0,"950 TOWER LANE, 6TH FLOORFOSTER CITY, CA 94404",CA,...,https://www.nasdaq.com/markets/ipos/company/qu...,2010-01-01,15.000000,15.00,16.410000,16.010000,12.980000,13.500000,12.740000,12.52
TRNO,TERRENO REALTY CORP,TRNO,New York Stock Exchange,20.0,8750000.0,175000000.0,2010-02-10,6.0,"16 MAIDEN LANEFIFTH FLOORSAN FRANCISCO, CA 94108",CA,...,https://www.nasdaq.com/markets/ipos/company/te...,2010-01-01,14.692022,18.75,15.330121,19.530001,14.770803,18.809999,13.518149,17.02
PDM,"PIEDMONT OFFICE REALTY TRUST, INC.",PDM,New York Stock Exchange,14.5,12000000.0,174000000.0,2010-02-10,109.0,"5565 GLENRIDGE CONNECTORSTE 450ATLANTA, GA 30342",GA,...,https://www.nasdaq.com/markets/ipos/company/pi...,2010-01-01,9.543714,14.75,12.046385,19.299999,10.400203,17.830000,14.289856,21.02


In [25]:
df_ipo.shape

(834, 21)

In [26]:
df_ipo.to_csv('data/ipo_stock_2010_2018_v2.csv', index=False)

## 1.2. Scraping profile and industry from Yahoo Finance

In this step, I scrape sector, industry, employees as of now, if available CEO pay and CEO year born from Yahoo Finance. 

In [27]:
sector = []
industry = []
employees2019 = []
CEO_pay = []
CEO_born = []

url_1 = 'https://finance.yahoo.com/quote/%s/'
url_2 = 'profile?p=%s'

for i, ticker in enumerate(df_ipo.Symbol):
    print(f"scraping {ticker} - {i+1}/{len(df_ipo.Symbol)}")
    result = requests.get(url_1 % ticker + url_2 % ticker)
    content = result.content

    soup = BeautifulSoup(content)
    data = soup.find_all('span', {"class":"Fw(600)"})
    if data:
        txt = [x.get_text() for x in data]
        sector.append(txt[0])
        industry.append(txt[1])
        employees2019.append(txt[2])

    else:
        sector.append(np.nan)
        industry.append(np.nan)
        employees2019.append(np.nan)
    
    try:
        table = pd.read_html(content)[0]
        #checking if table has CEO 
        if ('Title' in table) and (table['Title'].str.contains('CEO').sum() == 1):
            CEO_pay.append(table['Pay'][table['Title'].str.contains('CEO')].values[0]) 
            CEO_born.append(table['Year Born'][table['Title'].str.contains('CEO')].values[0])
        else:
            CEO_pay.append(np.nan) 
            CEO_born.append(np.nan)

    except ValueError:
        CEO_pay.append(np.nan) 
        CEO_born.append(np.nan)
    
print(f"checking lengths sector: {len(sector)}, industry: {len(industry)}, employees2019: {len(employees2019)}, CEO pay: {len(CEO_pay)}, CEO_born: {len(CEO_born)}")

scraping AMCF - 1/834
scraping GNRC - 2/834
scraping QNST - 3/834
scraping TRNO - 4/834
scraping PDM - 5/834
scraping IRWD - 6/834
scraping SSNC - 7/834
scraping STNG - 8/834
scraping HTHT - 9/834
scraping FIBK - 10/834
scraping CALX - 11/834
scraping MXL - 12/834
scraping AVEO - 13/834
scraping ST - 14/834
scraping AOSL - 15/834
scraping CDXS - 16/834
scraping ALIM - 17/834
scraping SPSC - 18/834
scraping CLDT - 19/834
scraping PRI - 20/834
scraping GNMK - 21/834
scraping JKS - 22/834
scraping EXPR - 23/834
scraping RRTS - 24/834
scraping TNAV - 25/834
scraping PLOW - 26/834
scraping TSLA - 27/834
scraping FN - 28/834
scraping OAS - 29/834
scraping GDOT - 30/834
scraping AMRC - 31/834
scraping WSR - 32/834
scraping ELMD - 33/834
scraping MMYT - 34/834
scraping RP - 35/834
scraping NXPI - 36/834
scraping AMRS - 37/834
scraping COR - 38/834
scraping SFUN - 39/834
scraping PACB - 40/834
scraping VRA - 41/834
scraping TOWR - 42/834
scraping EFC - 43/834
scraping DQ - 44/834
scraping STND 

scraping SPWH - 352/834
scraping LEJU - 353/834
scraping MC - 354/834
scraping PAYC - 355/834
scraping CIO - 356/834
scraping PAHC - 357/834
scraping FPI - 358/834
scraping ENBL - 359/834
scraping ADMS - 360/834
scraping ALLY - 361/834
scraping GRUB - 362/834
scraping FIVN - 363/834
scraping RUBI - 364/834
scraping PIH - 365/834
scraping RESN - 366/834
scraping PE - 367/834
scraping AGRX - 368/834
scraping HRTG - 369/834
scraping SDPI - 370/834
scraping JD - 371/834
scraping TGEN - 372/834
scraping TRUE - 373/834
scraping JMEI - 374/834
scraping BWFG - 375/834
scraping ZEN - 376/834
scraping SFBS - 377/834
scraping TOUR - 378/834
scraping PBFX - 379/834
scraping CMCM - 380/834
scraping LPG - 381/834
scraping ALDR - 382/834
scraping GLOP - 383/834
scraping SCYX - 384/834
scraping FRSH - 385/834
scraping ARES - 386/834
scraping MIK - 387/834
scraping NEP - 388/834
scraping SERV - 389/834
scraping GPRO - 390/834
scraping AMPH - 391/834
scraping MTLS - 392/834
scraping XNET - 393/834
scrap

scraping CLPR - 698/834
scraping FBM - 699/834
scraping SACH - 700/834
scraping KRP - 701/834
scraping INVH - 702/834
scraping AYX - 703/834
scraping VLRX - 704/834
scraping PUMP - 705/834
scraping ARD - 706/834
scraping PSDO - 707/834
scraping JILL - 708/834
scraping BYSI - 709/834
scraping SNAP - 710/834
scraping HLNE - 711/834
scraping CLDR - 712/834
scraping CVNA - 713/834
scraping EEX - 714/834
scraping NCSM - 715/834
scraping XRF - 716/834
scraping ZYME - 717/834
scraping VRNA - 718/834
scraping CATS - 719/834
scraping TOCA - 720/834
scraping CADE - 721/834
scraping HCC - 722/834
scraping YEXT - 723/834
scraping NETS - 724/834
scraping OKTA - 725/834
scraping SNDR - 726/834
scraping ELVT - 727/834
scraping HESM - 728/834
scraping APPN - 729/834
scraping WOW - 730/834
scraping SGH - 731/834
scraping ARGX - 732/834
scraping BEDU - 733/834
scraping GTHX - 734/834
scraping SOI - 735/834
scraping ASV - 736/834
scraping GDI - 737/834
scraping VERI - 738/834
scraping GNTY - 739/834
scra

In [28]:
employees2019_clean = []
for emp in employees2019:
    if emp is not np.nan:
        if len(emp) > 0:
            emp = int(emp.replace(',', ''))
        else:
            emp = np.nan
    employees2019_clean.append(emp)

In [29]:
CEO_pay_clean = []
for i, cp in enumerate(CEO_pay):
    if cp == cp:
        if isinstance(cp, float):
            print(i, cp)
            cp_clean = cp*1000
        elif 'M' in cp:
            print(i, cp)
            cp_clean = float(cp.replace("M", ""))*1000000
        elif 'k' in cp:
            print(i, cp)
            cp_clean = float(cp.replace("k", ""))*1000        
        CEO_pay_clean.append(cp_clean)
    elif cp != cp:
        print(i, 'this one', cp)
        CEO_pay_clean.append(cp)

0 this one nan
1 2.52M
2 816.15k
3 799.5k
4 this one nan
5 this one nan
6 10.88M
7 this one nan
8 this one nan
9 1.71M
10 500k
11 554.71k
12 770.17k
13 1.57M
14 1.34M
15 1.15M
16 501.12k
17 1.39M
18 2.39M
19 2.39M
20 1M
21 this one nan
22 this one nan
23 839.92k
24 821.3k
25 854.23k
26 56.38k
27 700.01k
28 1.84M
29 1.88M
30 1.17M
31 619.25k
32 418.35k
33 this one nan
34 1.38M
35 4.45M
36 1.8M
37 1.52M
38 this one nan
39 1.03M
40 1.4M
41 2.11M
42 this one nan
43 this one nan
44 this one nan
45 this one nan
46 978.89k
47 this one nan
48 869.7k
49 7.36M
50 3.26M
51 this one nan
52 6.01M
53 this one nan
54 461.26k
55 this one nan
56 13.69k
57 2.25M
58 1.47M
59 2.53M
60 2.22M
61 856.64k
62 this one nan
63 3.1M
64 this one nan
65 this one nan
66 1.82M
67 941k
68 908.96k
69 1.05M
70 1.68M
71 790.22k
72 429.17k
73 638.34k
74 252.62k
75 375.72k
76 1.67M
77 1.15M
78 3.84M
79 this one nan
80 3.52M
81 this one nan
82 this one nan
83 this one nan
84 1.66M
85 this one nan
86 4.4M
87 1M
88 1.55M
89 t

# Final DataFrame

In [30]:
df_ipo['sector'] = sector
df_ipo['industry'] = industry
df_ipo['employees2019'] = employees2019_clean
df_ipo['CEO_pay'] = CEO_pay_clean
df_ipo['CEO_born'] = CEO_born

In [31]:
df_ipo.head()

,Company Name,Symbol,Market,Price,Shares,Offer Amount,Date Priced,employees,address,US_state,...,inmonth_open,inweek_adjclose,inweek_open,inyear_adjclose,inyear_open,sector,industry,employees2019,CEO_pay,CEO_born
AMCF,ANDATEE CHINA MARINE FUEL SERVICES CORP,AMCF,NASDAQ,6.3,3134921.0,19750002.0,2010-01-26,128.0,NO. 68 BINHAI RD DALIAN XIGANG DISTRICTDALIAN ...,NaN,...,8.000000,5.830000,6.030000,2.720000,2.58,Energy,Oil & Gas Equipment & Services,189.0,NaN,NaN
GNRC,GENERAC HOLDINGS INC.,GNRC,New York Stock Exchange,13.0,18750000.0,243750000.0,2010-02-11,1486.0,"S45 W29290 HIGHWAY 59WAUKESHA, WI 53187",WI,...,14.910000,8.612183,13.050000,13.415763,20.24,Industrials,Diversified Industrials,5046.0,2520000.0,1972.0
QNST,"QUINSTREET, INC",QNST,NASDAQ,15.0,10000000.0,150000000.0,2010-02-11,568.0,"950 TOWER LANE, 6TH FLOORFOSTER CITY, CA 94404",CA,...,16.010000,12.980000,13.500000,12.740000,12.52,Technology,Internet Content & Information,506.0,816150.0,1960.0
TRNO,TERRENO REALTY CORP,TRNO,New York Stock Exchange,20.0,8750000.0,175000000.0,2010-02-10,6.0,"16 MAIDEN LANEFIFTH FLOORSAN FRANCISCO, CA 94108",CA,...,19.530001,14.770803,18.809999,13.518149,17.02,Real Estate,REIT - Industrial,23.0,799500.0,1961.0
PDM,"PIEDMONT OFFICE REALTY TRUST, INC.",PDM,New York Stock Exchange,14.5,12000000.0,174000000.0,2010-02-10,109.0,"5565 GLENRIDGE CONNECTORSTE 450ATLANTA, GA 30342",GA,...,19.299999,10.400203,17.830000,14.289856,21.02,Real Estate,REIT - Office,134.0,NaN,NaN


In [32]:
df_ipo.shape

(834, 26)

In [33]:
df_ipo.to_csv('data/ipo_stock_2010_2018_v2.csv', index=False)

## Creating dill to serialize python objects 

In [5]:
import sys
print(sys.getrecursionlimit())
sys.setrecursionlimit(10000)
print(sys.getrecursionlimit())

3000
10000


In [ ]:
dill.dump_session('dill/ipo_price_scraper.pkl')